# Çoklu Doğrusal Regresyon (Multiple Linear Regression) İle Nem Tahmini

### Gerekli Kütüphaneler

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
# verinin yüklenmesi
df = pd.read_csv("odev_tenis.csv")

In [3]:
# verinin ilk 5 satırı  
df.head()

,outlook,temperature,humidity,windy,play
0,sunny,85,85,False,no
1,sunny,80,90,True,no
2,overcast,83,86,False,yes
3,rainy,70,96,False,yes
4,rainy,68,80,False,yes


### İlk 5 satırın türkçeleştirilmiş hali

<table style="float:left">
  <tr>
    <th> </th>
    <th>havaŞartı</th>
    <th>sıcaklık</th>
    <th>nem</th>
    <th>rüzgarlı</th>
    <th>oynanır</th>
  </tr>
  <tr>
    <td>0</td>
    <td>güneşli</td>
    <td>85</td>
    <td>85</td>
    <td>Yanlış</td>
    <td>hayır</td>
  </tr>
  <tr>
    <td>1</td>
    <td>güneşli</td>
    <td>80</td>
    <td>90</td>
    <td>Doğru</td>
    <td>hayır</td>
  </tr>
  <tr>
    <td>2</td>
    <td>bulutlu</td>
    <td>83</td>
    <td>86</td>
    <td>Yanlış</td>
    <td>evet</td>
  </tr>
  <tr>
    <td>3</td>
    <td>yağmurlu</td>
    <td>70</td>
    <td>96</td>
    <td>Yanlış</td>
    <td>evet</td>
  </tr>
  <tr>
    <td>4</td>
    <td>yağmurlu</td>
    <td>68</td>
    <td>80</td>
    <td>Yanlış</td>
    <td>evet</td>
  </tr>
</table>


In [4]:
# verinin satır ve sutun sayısı
df.shape

(14, 5)

Veri seti 14 satırdan ve 5 sutundan oluşuyor

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   outlook      14 non-null     object
 1   temperature  14 non-null     int64 
 2   humidity     14 non-null     int64 
 3   windy        14 non-null     bool  
 4   play         14 non-null     object
dtypes: bool(1), int64(2), object(2)
memory usage: 590.0+ bytes


Veri seti 1 adet boolean (mantıksal->True,False), 2 adet integer (sayısal), 2 adet de object (kategorik) tipinde değişkenden oluşuyor.

## Veri Ön İşleme
Makine öğrenmesi modelleri sadece sayısal verilerle işlem yapabilirler dalayısıyla kuracağımız modelin daha anlamlı çıktılar üretebilmesi için bool ve object tipindeki sutunların sayısal hale dönüşütürülmesi gerekir.

## Label Encoding
windy ve play sutunlarını label encoding işlemine tabi tutacağız bu sayade veri tipleri binary yani 0-1 haline dönüşmüş olacaktır. 

In [6]:
# gerekli fonksiyonun çağrılması
from sklearn.preprocessing import LabelEncoder

In [7]:
# label encoder objesi oluşturduk
le = LabelEncoder()

In [8]:
df["windy"] = le.fit_transform(df["windy"]) # windy dönüşümü

In [9]:
df["play"] = le.fit_transform(df["play"]) # play dönüşümü

In [10]:
df.head()

,outlook,temperature,humidity,windy,play
0,sunny,85,85,0,0
1,sunny,80,90,1,0
2,overcast,83,86,0,1
3,rainy,70,96,0,1
4,rainy,68,80,0,1


Dönüşümden sonda `False` ve `no` değerleri 0 olurken `True` ve `yes` değerleri 1 oldu.

## One-Hot Encoding
outlook sutununu one-hot encoding ile dönüştüreceğiz çünkü hava şartlarının kendi aralarında sınıfsal bir farkı yoktur.

In [11]:
# gerekli fonksiyonun çağrılması
from sklearn.preprocessing import OneHotEncoder

In [12]:
df = pd.concat([df, pd.get_dummies(df["outlook"], prefix="outlook")], axis=1).drop("outlook", axis=1) # outlook dönüşümü

In [13]:
df.head()

,temperature,humidity,windy,play,outlook_overcast,outlook_rainy,outlook_sunny
0,85,85,0,0,0,0,1
1,80,90,1,0,0,0,1
2,83,86,0,1,1,0,0
3,70,96,0,1,0,1,0
4,68,80,0,1,0,1,0


## Veri Kontrolü

In [14]:
# eksik veri var mı
df.isnull().sum()

temperature         0
humidity            0
windy               0
play                0
outlook_overcast    0
outlook_rainy       0
outlook_sunny       0
dtype: int64

In [15]:
# veri tipleri
df.dtypes

temperature         int64
humidity            int64
windy               int64
play                int32
outlook_overcast    uint8
outlook_rainy       uint8
outlook_sunny       uint8
dtype: object

Veri setimizde eksik veri ve sayısal olamayan değer yok. Model kurulum işlemine geçebiliriz.

## Model Kurulumu

humidity kolonu bağımlı değişkenimiz olacak diğer kolonlar ise bağımsız değişkenlerimiz olacaklar.

In [16]:
# rahat işlem yapabilmek için humidity kolonunu sona alalım
df = df[["windy", "play","outlook_overcast", "outlook_rainy", "outlook_sunny", "temperature", "humidity"]]

In [17]:
# gerekli fonksiyonun çağrılması
from sklearn.model_selection import train_test_split

In [18]:
# Verinin train ve test olarak ikiye bölünmesi
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, :-1], df.iloc[:, -1], test_size=0.33, random_state=0)

Veri setimizin $2/3$'ünü eğitim için kalan $1/3$'ünü de test için ayırıyoruz 

In [19]:
# esasen veri setimiz 4 parçaya bölündü 
print(X_train.to_markdown())
print(X_test.to_markdown())
print(y_train.to_markdown())
print(y_test.to_markdown())

|    |   windy |   play |   outlook_overcast |   outlook_rainy |   outlook_sunny |   temperature |
|---:|--------:|-------:|-------------------:|----------------:|----------------:|--------------:|
| 13 |       1 |      0 |                  0 |               1 |               0 |            71 |
|  9 |       0 |      1 |                  0 |               1 |               0 |            75 |
|  1 |       1 |      0 |                  0 |               0 |               1 |            80 |
|  7 |       0 |      0 |                  0 |               0 |               1 |            72 |
| 10 |       1 |      1 |                  0 |               0 |               1 |            75 |
|  3 |       0 |      1 |                  0 |               1 |               0 |            70 |
|  0 |       0 |      0 |                  0 |               0 |               1 |            85 |
|  5 |       1 |      0 |                  0 |               1 |               0 |            65 |
| 12 |    

### Çoklu Doğrusal Regresyon

In [20]:
# gerekli fonksiyonun çağrılması
from sklearn.linear_model import LinearRegression

In [21]:
# linear regression objesi oluşturduk
regressor = LinearRegression()

In [22]:
# model eğitimi
regressor.fit(X_train,y_train)
y_pred = regressor.predict(X_test)

In [23]:
# tahmin değerlerimiz
print(y_pred)

[84.45365573 63.93839954 85.76050662 64.21013241 75.06793322]


In [24]:
tahmin = pd.DataFrame(y_pred, columns=["tahmin"])

result = pd.concat([y_test.reset_index(drop=True), tahmin], axis=1)
print(result)

   humidity     tahmin
0        70  84.453656
1        65  63.938400
2        80  85.760507
3        90  64.210132
4        86  75.067933


## Backward Elimination (geriye doğru eleme)
H0: İki değişken arasında ilişki yokur.  -> Yokluk hipotezi

H1: İki değişken arasında ilişki vardır. -> Alternatif hipotez

Alfa standart olarak 0.05 alınır.        -> $α = 0.05$

P-value alfadan büyükse H0 reddedilemez. -> $p > α$ H0'ı reddetmek için yeterli delil yok

P-value alfadan küçükse H0 reddedilir.   -> $p < α$ H0 red

Regresyon modeli kurarken, bağımlı değişkenlerle ilişkisi yüksek olan bağımsız değişkenlerin seçilmesi amaçlanır dolayısıyla H0 hipotezini reddetmeliyiz.

In [25]:
# gerekli kütüphanenin çağrılması
import statsmodels.api as sm

In [26]:
# model eğitimi
X = np.append(arr = np.ones((14,1)).astype(int), values=df.iloc[:,:-1], axis=1 )
X_l = df.iloc[:,[0,1,2,3,4,5]].values
r_ols = sm.OLS(endog = df.iloc[:,-1:], exog =X_l)
r = r_ols.fit()
r.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               humidity   R-squared:                       0.294
Model:                            OLS   Adj. R-squared:                 -0.148
Method:                 Least Squares   F-statistic:                    0.6653
Date:                Sat, 04 Feb 2023   Prob (F-statistic):              0.661
Time:                        21:28:20   Log-Likelihood:                -49.542
No. Observations:                  14   AIC:                             111.1
Df Residuals:                       8   BIC:                             114.9
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -4.0286      7.229     -0.557      0.593     -20.698      12.641
x2            -8.2778      8.029     -1.031      0.333     -26.793      10.237
x3            52.3891     50.214      1.043      0.327     -63.404     168.183
x4            55.6337     45.233      1.230      0.254     -48.673     159.940
x5            49.4291     48.337      1.023      0.336     -62.035     160.893
x6             0.4920      0.597      0.825      0.433      -0.884       1.868
==============================================================================
Omnibus:                        0.935   Durbin-Watson:                   2.416
Prob(Omnibus):                  0.627   Jarque-Bera (JB):                0.823
Skew:                           0.389   Prob(JB):                        0.663
Kurtosis:                       2.103   Cond. No.                     2.08e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.08e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [27]:
# p-value yani olasılık değerleri
r.pvalues

x1    0.592569
x2    0.332708
x3    0.327300
x4    0.253657
x5    0.336421
x6    0.433415
dtype: float64

Backward Elimination yöntemi ile kurduğumuz modelin tahmin başarısını arttırmak için sisteme etkisi az olan bağımız değişkenleri atacağız.

P-value değerlerine baktığımız zaman olasılığı en yüksek olan x1'dir. Dolayısıyla onu sistemden atacağız.

In [28]:
# en yüksek olasılık değerine sahip x1'yi yani windy sutununu attık
df = df.iloc[:,1:]

In [29]:
# tekrardan modelimizi kuralım
X = np.append(arr = np.ones((14,1)).astype(int), values=df.iloc[:,:-1], axis=1 )
X_l = df.iloc[:,[0,1,2,3,4]].values
r_ols = sm.OLS(endog = df.iloc[:,-1:], exog =X_l)
r = r_ols.fit()
r.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               humidity   R-squared:                       0.266
Model:                            OLS   Adj. R-squared:                 -0.060
Method:                 Least Squares   F-statistic:                    0.8165
Date:                Sat, 04 Feb 2023   Prob (F-statistic):              0.546
Time:                        21:28:21   Log-Likelihood:                -49.809
No. Observations:                  14   AIC:                             109.6
Df Residuals:                       9   BIC:                             112.8
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -6.2865      6.909     -0.910      0.387     -21.916       9.343
x2            36.6564     39.905      0.919      0.382     -53.615     126.927
x3            41.9135     36.464      1.149      0.280     -40.573     124.400
x4            35.1064     39.339      0.892      0.395     -53.884     124.097
x5             0.6484      0.506      1.282      0.232      -0.496       1.793
==============================================================================
Omnibus:                        0.887   Durbin-Watson:                   2.360
Prob(Omnibus):                  0.642   Jarque-Bera (JB):                0.810
Skew:                           0.424   Prob(JB):                        0.667
Kurtosis:                       2.181   Cond. No.                     1.73e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.73e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [30]:
# yeni modeli eğitelim ve tekrardan tahmin yapalım
X_train = X_train.iloc[:,1:]
X_test = X_test.iloc[:,1:]

regressor.fit(X_train,y_train)
y_pred2 = regressor.predict(X_test)

In [31]:
print(y_pred2)

[77.98135142 68.29304916 81.05037539 71.44926132 75.78905304]


In [32]:
tahmin2 = pd.DataFrame(y_pred2, columns=["tahmin_2"])

result2 = pd.concat([y_test.reset_index(drop=True), tahmin2], axis=1)
print(result2)

   humidity   tahmin_2
0        70  77.981351
1        65  68.293049
2        80  81.050375
3        90  71.449261
4        86  75.789053


### Eski tahmin değerleri ile Backward Elimination yöntemi uygulandıktan sonraki tahmin değerlerinin karşılaştırılması

In [33]:
result_df = pd.concat([result, tahmin2], axis=1)
result_df

,humidity,tahmin,tahmin_2
0,70,84.453656,77.981351
1,65,63.938400,68.293049
2,80,85.760507,81.050375
3,90,64.210132,71.449261
4,86,75.067933,75.789053


Sonuçları birbirleri ile kıyasladığımızda modelimizin daha isabetli tahminler yapabildiğini söyleyebiliriz.